IZENA: Oier

ABIZENAK: Alvarez

# Eguzki sistemaren 5 gorputzeko eredua

Eguzki sistemaren 5 gorputzeko sistema batekin egingo dugu lan. Bertan, bost masa puntual Newton-en grabitate legearen arabera mugituko dira: Eguzkia, Jupiter, Saturno, Urano, eta Neptuno.

<ul id="top">

<a href="#top">Ir al inicio</a>

## Paketeak eta funtzioak kargatu

In [2]:
import Pkg
Pkg.add("LinearAlgebra")
Pkg.add("OrdinaryDiffEq")
Pkg.add("Polynomials")
using LinearAlgebra
using Plots
using OrdinaryDiffEq
using Polynomials

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed Qt5Base_jll ─ v5.15.3+2


    Updating `~/.julia/environments/v1.8/Project.toml`
  [37e2e46d] + LinearAlgebra
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [ea2cea3b] ↑ Qt5Base_jll v5.15.3+1 ⇒ v5.15.3+2


Precompiling project...


...

  ✓ OrdinaryDiffEq
  91 dependencies successfully precompiled in 262 seconds. 134 already precompiled.
   Resolving package versions...


   Installed Polynomials ─ v3.2.0


    Updating `~/.julia/environments/v1.8/Project.toml`
  [f27b6e38] + Polynomials v3.2.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [f27b6e38] + Polynomials v3.2.0


Precompiling project...


  ✓ Polynomials
  1 dependency successfully precompiled in 2 seconds. 225 already precompiled.


In [ ]:
include("KanpokoEguzkiSistema.jl")



## 1 - $N$ gorputzeko eredua matematikoaren definizioa


Sistemaren energia, $q_1,\ldots,q_N \in \mathbb{R}^3$  kokapenen eta  $v_1,\ldots,v_N \in \mathbb{R}^3$  abiadura bektoreen funtzio gisa honakoa da:
\begin{equation*}
\begin{split}
\mathrm{Energy}(q_1,\ldots,q_N,v_1,\ldots,v_N) = 
\frac12 \sum_{i=1}^{N} m_i \, \|v_i\|^2
- G \sum_{1\leq i < j \leq N} \frac{m_i m_j}{\|q_i-q_j\|}.
\end{split}
\end{equation*}


Ekuazio diferentzialen sistema:
\begin{equation}
\begin{split}
\frac{d}{dt} q_i &= v_i, \\
\frac{d}{dt} v_i &= G\, 
\sum_{j \neq i} \frac{m_j}{\|q_j-q_i\|^3}\,
(q_j-q_i).
\end{split} \tag{1}
\end{equation}

Sistema hori modu trinkoan horrela adieraziko dugu:

\begin{equation*}
\frac{d u}{dt} = f(t,u,p). \tag{2}
\end{equation*}



Egoera aldagaiak, bektore batean bildu beharrean, hiru indizeko array batean horrela gordeko ditugu:

\begin{equation*}
u = 
\left(\begin{matrix}
q_1  & v_1\\
\vdots & \vdots \\
q_N  & v_N
\end{matrix}
\right) \in \mathbb{R}^{3\times N \times 2}.
\end{equation*}

Aldiz, parametro konstanteen bektorea honakoa da:

\begin{equation*}
p = (G\,  m_1, \ldots,G\,  m_N) \in \mathbb{R}^N.
\end{equation*}

**Unitate neurriak:** distantzia unitate astronomikotan (AU), eta denbora egunetan.

### 1.1 NbodyEnergy(u,p) funtzioa inplementatu, sistemaren energia bider $G$ kalkulatzen duena

In [ ]:
function NbodyEnergy(u,p)
     N = length(p)
     T = 0.
     U = 0.
     for i in 1:N
        qi = u[:,i,1]
        vi = u[:,i,2]
        Gmi = p[i]
        T += Gmi*dot(vi,vi)
        for j in (i+1):N
           qj = u[:,j,1]
           Gmj = p[j]
           qij = qi - qj
           U -= ?
        end
     end
    return ?
end


### 1.2 $N$ gorputzeko problema (2) formatuan jartzeko behar den NbodyODE!(du,u,p,t) funtzioa inplementatu. 

In [ ]:
function NbodyODE!(du,u,p,t)
     # Hemen, du taulan jarriko dira $f(t,u,p)$ aplikazioaren osagaiak. 
     N = length(p)
     du[:,:,2] .= 0
     for i in 1:N
        xi = u[1,i,1]
        yi = u[2,i,1]
        zi = u[3,i,1]
        Gmi = p[i] 
        for j in (i+1):N
           xij = xi - u[1,j,1]
           yij = yi - u[2,j,1]
           zij = zi - u[3,j,1]
           Gmj = p[j] 
           dotij = xij^2+yij^2+zij^2
           ... 
           auxij = ? 
           du[1,i,2] -= Gmj*auxij*xij
           du[1,j,2] += Gmi*auxij*xij
           du[2,i,2] -= Gmj*auxij*yij
           du[2,j,2] += Gmi*auxij*yij
           du[3,i,2] -= Gmj*auxij*zij
           du[3,j,2] += Gmi*auxij*zij
        end
     end
    du[:,:,1] .= ? 
    return nothing
end







##### <a href="#top">Hasierara</a>

## 2 - Eguzki sistemaren bilakaeraren datu "errealak"

KanpokoEguzkiSistemarenDatuak(tend) funtzioak kanpoko eguzki-sistemaren datu "errealak" itzultzen dizkigu $[0,t_{end}]$ denbora-tartearen diskretizazio uniforme jakin baterako. Hemen, $t_{end}=60182$ da, Neptunoren periodoa egunetan.

Datu horiek lortzeko, 16 gorputzeko eredu errealistago baten simulazioa egin da: eguzkia, eguzki-sistemako zortzi planetak, ilargia, Pluto, eta Ceres, Pallas, Vesta, Iris eta Bamberga (Marte eta Jupiterren arteko asteroide gerrikoko bost objektu handienak). Hasierako datuak 1969ko ekainaren 28koak dira (Julian Day (TDB) 2440400.5), DE430 efemerideetatik lortutakoak.

In [ ]:
tend= 60182. # Neptunoren gutxi gorabeherako periodoa egunetan
datos_t, datos_u = KanpokoEguzkiSistemarenDatuak(tend);

### 2.1 - Hiru irudi lortu, OXY, OXZ eta OYZ planoetako proiekzioei dagozkienak, hurrenez hurren, kanpoko lau planeten orbiten grafikoekin.


In [ ]:
pl = plot(title="Orbiten proiekzioa OXY planoaren gainean", 
            xlabel="x", ylabel="y", legend=false, aspect_ratio=1)

  for j = 1:5
    x  = [? for u in datos_u]
    y  = [? for u in datos_u] 
    pl = plot!(x,y) 
  end 
display(pl)

In [ ]:
pl = plot(title="Orbiten proiekzioa OXZ planoaren gainean", 
            xlabel="x", ylabel="z", legend=false, aspect_ratio=1)

  for j = 1:5
    x  = [? for u in datos_u]
    z  = [? for u in datos_u] 
    pl = plot!(x,z) 
  end 
display(pl)

In [ ]:
 pl = plot(title="Orbiten proiekzioa OYZ planoaren gainean", 
            xlabel="y", ylabel="z", legend=false, aspect_ratio=1)

  for j = 1:5
    y  = [? for u in datos_u]
    z  = [? for u in datos_u] 
    pl = plot!(y,z) 
  end 
display(pl)

##### <a href="#top">Hasierara</a>

## 3 - Kanpoko eguzki sistemaren eredu matematikoaren soluzioa

KanpokoEguzkiSistemarenHasierakoDatuak() honakoa itzultzen digu:

- Sistemaren hasierako $u_0$ egoera 1969ko ekainaren 28an (Julian Day (TDB) 2440400.5), DE430 efemerideetatik lortuak), 
- Parametro konstanteen osagaien $p_i = G\, m_i$  ($i=1,2,3,4,5$) balioak, eta 
- Gorputz bakoitzaren izenen zerrenda. 

In [ ]:
u0, p, bodylist = KanpokoEguzkiSistemarenHasierakoDatuak()
GmNeptuno = p[5]
N = length(p)
bodylist

Neptunori dagokion $G\, m_5 $-ren balioa 1992an balio hori zuzendu aurretik zegoena da.
(1989an Neptunoren orbitatik gertu Voyager2 pasa zenean jasotako datuekin zuzendu zen Neptunoren masa.)

### 3.1 - Bost gorputzen problemaren emaitzak lortu  datos_t taulan emandako $t_1,\ldots, t_m $ denboretarako, horretarako Vern9() metodoa aplikatuz  $h = 300$ urrats luzerarekin

In [ ]:
tspan = (0.,datos_t[end])
prob = ?
h = 300.
sol = ?

### 3.2 - Energiaren errore erlatiboaren bilakaera irudikatu eskala logaritmikoan


In [ ]:
?
?
pl = plot(sol.t, EE, xlabel="t", title="Energia-errorea", 
                     yscale=:log10, ylims=(1e-17,1e-8),
                     label="h=$h", legend=:right )

### 3.3 -  Kalkuluak errepikatu $h = 150$ urrats-luzerarekin, eta gehitu aurreko irudiari emaitza berrien energia-errore erlatiboaren bilakaeraren grafikoa. Gehitu, halaber, energia-errore-erlatiboaren grafikoa $h = 75$ urrats-luzeraren kasuan


In [ ]:
    h /= 2
    sol2 = ?
    EE = ?
    plot!(sol.t, EE, label="h=$h", legend=:right )

    h /= 2
    sol2 = ?
    EE = ?
    plot!(sol.t, EE, label="h=$h", legend=:right )


### 3.4 Alde batetik datu errealak  eta bestetik zenbakizko integrazioarekin lortutako emaitzak hartuz irudi bakar batean irudikatu planeta bakoitzaren kokapenen diferentzien normaren bilakaera (eskala logaritmikoan)


In [ ]:
...

### 3.5 Vern9 metodoarekin eta $h=150$ urrats luzerarekin kalkulatutako emaitzen posizioetako zenbakizko erroreak estimatu (bigarren gaiko amaierako teknika erabiliz), eta 3.4 ataleko irudi berean gehitu errore horien normaren bilakaeraren grafikoak (eskala logaritmikoan).


In [ ]:
...

In [ ]:
...

### 3.6 Vern9 metodoarekin eta $h=150$ urrats luzerarekin kalkulatutako emaitzen posizioetako zenbakizko errore estimatuen tamainia ikusirik, 3.4 atalean lortutako grafikoa fidagarria dela esango zenuke?


>
> Erantzuna hemen idatzi

##### <a href="#top">Ir al inicio</a>

## 4 - Neptunoren masaren doikuntza

Atal honetan, $p_5 = G\, m_5$ parametroa doitzen saiatuko gara, parametroaren balio horretarako lortutako emaitza simulatuak, gutxieneko karratuen zentzuan, datu errealekin (ahal den neurrian)  bat etor daitezen.


### 4.1 - Errfcn (x) funtzio bat definitu, parametroaren  $x = G\, m_5$ balioari dagokion hondarra. Hau da, lau planeten posizioen osagai bakoitzaren diferentzien karratuen batura, datu errealen eta (Vern9 metodoarekin $h=150$ urrats luzera erabiliz) kalkulatuta emaitzen arteko desadostasunaren neurri gisa 

In [ ]:
p_= copy(p)
N = length(p)
prob_ = ODEProblem(NbodyODE!, u0, tspan, p_)

function errfcn(x)
    p_[5] = x
    h = 300.
    ...
    return norm2err
end

### 4.2 errfcn(x) funtzioaren grafikoa irudikatu,  $[0.9 G m_5, 1.1 G m_5]$ tartean uniformeki banatutako $x$-en 11 baliotarako

In [ ]:
xa = 0.9*p[5]
xb = 1.1*p[5]
xrange = ?
errores = ?

scatter(xrange, errores, legend=false)

### 4.3 -  Lortu $x$-ren balio horien artean $x = x_ {min}$ balioa, errfcn (x) hondarraren baliorik txikiena hartzen duena, eta alderatu balio hori $x = G\, m_5$-rekin lortzen zenarekin.

In [ ]:
...
xmin = ?

(errfcn(p[5]), errfcn(xmin))

### 4.4 - Errepikatu orain 3.4 ataleko grafikoa, baina $G\, m_5$ erabili beharrean, $p_5=x_ {min}$ hartuta

In [ ]:
...

In [ ]:
...

Oraingoan, 3.4 ataleko diferentzien aldean, diferentzia txikiagoak lortu beharko lirateke. Alegia, masaren balio berriarekin, emaitza simulatuak datu errealetara hobe doitzen dira.

## 5 - Neptunoren masaren doikuntza zehatzagoa

Hemen, $G\, m_5 $-ren balioaren doikuntza zehatzagoa egiteko,

- Errfcn(x) funtzioa $x = x_\min $-ren inguruan hurbilduko dugu 2. mailako $P_2 (x)$ polinomio baten bidez, errfcn(x) funtzioarekin bat datorrena $x$ aldagaiaren ondoko hiru balioetan: $x_\min$, eta $x_\min$-en gertuen dituen xrange-ko bi $x$-en balioak.

- Errfcn(x) minimizatzen duen $x$-ren balioa honela hurbilduko dugu: $P '_2 (x) = 0$ ekuazioa askatuko dugu. Horrela lortutako $x$-ren balio horrek $P_2(x)$ minimizatzen duela egiaztatuko dugu, eta beraz, errfcn(x) gutxi gorabehera minimizatzen duela.

### 5.2 - Polynomials paketeko fit funtzioa erabiliz, kalkulatu $P_2 (x)$ polinomio interpolatzailea. Eskatutako hiru interpolazio-baldintzak betetzen dituela egiaztatu.

In [ ]:
...

### 5.3 - Kalkulatu $x=x_\min^*$ non $P_2'(x)=0$ den
_Oharra_: Polynomials paketearekin lortutako P2 polinomioaren koefizienteen bektorea P2.coeffs modual lor daiteke.

In [ ]:
xmin_ = ?

### 5.4 - Konparatu $x = x_\min^*$ berri honen errfcn(x) balioa (4.3 atalean lortutako) errfcn($x_\min$) balioarekin


In [ ]:
(errfcn(xmin_)), errfcn(xmin)

Oharra: 5.3 atalean lortutako $x =x_\min^*$  balio optimorako errfcn(x) balioa errfcn($x_\min$) baino zertxobait txikiagoa izan beharko litzateke.

### 5.5 Errepikatu orain 3.4 eta 4.4 ataletako grafikoak, baina oraingoan $p_5 = x_\min^*$-ren balio berriari dagokion simulazioarekin

In [ ]:
...

### 5.6 Datu errealetara hobeto egokitzen al da eredu matematikoaren soluzioa $p_5= x_\min^*$ balioarekin?